In [ ]:
from pydantic import BaseModel

from daft_func import Pipeline, Runner, func


# 1. Define your data models
class Query(BaseModel):
    id: str
    text: str


class Result(BaseModel):
    id: str
    score: float


@func(output="results", map_axis="query", key_attr="id")
def process(query: Query, threshold: float) -> Result:
    score = len(query.text) * threshold
    return Result(id=query.id, score=score)


# 3. Create pipeline and runner
pipeline = Pipeline(functions=[process])
runner = Runner(pipeline=pipeline, mode="auto")

In [4]:
outputs = runner.run(
    inputs={
        "query": [Query(id="q1", text="hello")],
        "threshold": 0.5,
    }
)

print(outputs["results"])
# [Result(id='q1', score=2.5), Result(id='q2', score=2.5)]

[Result(id='q1', score=2.5)]


In [ ]:
outputs = runner.run(
    inputs={
        "query": [
            Query(id="q1", text="hello"),
            Query(id="q2", text="world"),
        ],
        "threshold": 0.5,
    }
)

print(outputs["results"])
# [Result(id='q1', score=2.5), Result(id='q2', score=2.5)]